# 1-1. Object Detection Demo
## 이미지에 보이는 오브젝트 인식

In [ ]:
%pip install google-cloud-vision --user
%pip install pillow --user

### Authentication

In [ ]:
from google.oauth2.service_account import Credentials
credentials = Credentials.from_service_account_file('auth.json')

### Cloud Vision API call
#### https://cloud.google.com/vision/docs/object-localizer

In [ ]:
from PIL import Image, ImageDraw

def draw_line(image, x1, y1, x2, y2, color="lime", width=2):
    draw = ImageDraw.Draw(image)
    draw.line((x1, y1, x2, y2), fill=color, width=width)
    #print (f"{x1},{y1} -> {x2}, {y2}")

    return image

from google.cloud import vision
def localize_objects(path):
    """Localize objects in the local image.

    Args:
    path: The path to the local file.
    """
    output_image = Image.open(path)
    width, height = output_image.size
    client = vision.ImageAnnotatorClient(credentials=credentials)

    with open(path, "rb") as image_file:
        content = image_file.read()
    image = vision.Image(content=content)

    objects = client.object_localization(image=image).localized_object_annotations

    print(f"Number of objects found: {len(objects)}")
    for object_ in objects:
        print(f"{object_.name} (confidence: {round(object_.score,2)})")
        #print("Normalized bounding polygon vertices: ")
        cnt = 0
        start_x = 0
        start_y = 0
        for vertex in object_.bounding_poly.normalized_vertices:
            print(f" - ({vertex.x}, {vertex.y})")
            if (cnt > 0):
                output_image = draw_line(output_image,
                                         start_x*width, start_y*height,
                                         vertex.x*width, vertex.y*height)
            else:
                init_x = vertex.x
                init_y = vertex.y
            cnt = cnt + 1
            start_x = vertex.x
            start_y = vertex.y

        output_image = draw_line(output_image,
                                         start_x*width, start_y*height,
                                         init_x*width, init_y*height)

    return output_image

def detect_logos(path):
    """Detects logos in the file."""
    from google.cloud import vision

    client = vision.ImageAnnotatorClient(credentials=credentials)

    with open(path, "rb") as image_file:
        content = image_file.read()

    image = vision.Image(content=content)

    response = client.logo_detection(image=image)
    print(response)
    logos = response.logo_annotations
    print("Logos:")

    for logo in logos:
        print(logo.description)

    if response.error.message:
        raise Exception(
            "{}\nFor more info on error messages, check: "
            "https://cloud.google.com/apis/design/errors".format(response.error.message)
        )

def detect_labels(path):
    """Detects labels in the file."""
    from google.cloud import vision

    client = vision.ImageAnnotatorClient(credentials=credentials)

    with open(path, "rb") as image_file:
        content = image_file.read()

    image = vision.Image(content=content)

    response = client.label_detection(image=image)
    print(response)
    labels = response.label_annotations
    print("Labels:")

    for label in labels:
        print(label.description)

    if response.error.message:
        raise Exception(
            "{}\nFor more info on error messages, check: "
            "https://cloud.google.com/apis/design/errors".format(response.error.message)
        )


### image sample - 01
#### [29cm] 다용도 식도 주방 칼
#### https://product.29cm.co.kr/catalog/2338303?category_large_code=292100100

In [ ]:
object_image = "object_detection_sample_01.jpg"
display(localize_objects(object_image))

### image sample - 02
#### [29cm] 발뮤다 New 더 스토리
#### https://product.29cm.co.kr/catalog/396529

#### 로고 인식

In [ ]:
object_image = "object_detection_sample_02.jpg"
display(Image.open(object_image))
detect_logos(object_image)

### 라벨 인식

In [ ]:
detect_labels(object_image)

## 도전과제 : Gemini 1.5 Pro API로 같은 기능을 하도록 만들기